In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.functions import *

#spark=SparkSession.builder.master("local").setApp("Linear Regression Example").getOrCreate()

df=spark.read.format("csv").options(inferSchema="true",header="true").load("/FileStore/tables/Linear_regression_dataset.csv")
print(df.show())
print("Getting the Shape of the Data")
print("Num of Rows:{0} Num of Columns:{1}".format(df.count(), len(df.columns)))
print("\nViewing the Schema of the Data")
print(df.printSchema)


+-----+-----+-----+-----+-----+-----+
var_1|var_2|var_3|var_4|var_5|label|
+-----+-----+-----+-----+-----+-----+
 734| 688| 81|0.328|0.259|0.418|
 700| 600| 94| 0.32|0.247|0.389|
 712| 705| 93|0.311|0.247|0.417|
 734| 806| 69|0.315| 0.26|0.415|
 613| 759| 61|0.302| 0.24|0.378|
 748| 676| 85|0.318|0.255|0.422|
 669| 588| 97|0.315|0.251|0.411|
 667| 845| 68|0.324|0.251|0.381|
 758| 890| 64| 0.33|0.274|0.436|
 726| 670| 88|0.335|0.268|0.422|
 583| 794| 55|0.302|0.236|0.371|
 676| 746| 72|0.317|0.265| 0.4|
 767| 699| 89|0.332|0.274|0.433|
 637| 597| 86|0.317|0.252|0.374|
 609| 724| 69|0.308|0.244|0.382|
 776| 733| 83|0.325|0.259|0.437|
 701| 832| 66|0.325| 0.26| 0.39|
 650| 709| 74|0.316|0.249|0.386|
 804| 668| 95|0.337|0.265|0.453|
 713| 614| 94| 0.31|0.238|0.404|
+-----+-----+-----+-----+-----+-----+
only showing top 20 rows

None
Getting the Shape of the Data
Num of Rows:1232 Num of Columns:6

Viewing the Schema of the Data
<bound method DataFrame.printSchema of DataFrame[var_1: int, var_2: int, var_3: int, var_4: double, var_5: double, label: double]>

In [2]:
print("\nViewing the Statistics of the Table")
print(df.describe().show())

print("Determing Co-Relation between Input and Output Columns")
print("Correlation between var_1 and label : {0}".format(df.select(corr('var_1','label')).show()))
print("Correlation between var_2 and label : {0}".format(df.select(corr('var_2','label')).show()))
print("Correlation between var_3 and label : {0}".format(df.select(corr('var_3', 'label')).show()))
print("Correlation between var_4 and label : {0}".format(df.select(corr('var_4', 'label')).show()))
print("Correlation between var_5 and label : {0}".format(df.select(corr('var_5', 'label')).show()))	  

Viewing the Statistics of the Table
+-------+-----------------+-----------------+------------------+--------------------+--------------------+-------------------+
summary| var_1| var_2| var_3| var_4| var_5| label|
+-------+-----------------+-----------------+------------------+--------------------+--------------------+-------------------+
 count| 1232| 1232| 1232| 1232| 1232| 1232|
 mean|715.0819805194806|715.0819805194806| 80.90422077922078| 0.3263311688311693| 0.25927272727272715|0.39734172077922014|
 stddev| 91.5342940441652|93.07993263118064|11.458139049993724|0.015012772334166148|0.012907228928000298|0.03326689862173776|
 min| 463| 472| 40| 0.277| 0.214| 0.301|
 max| 1009| 1103| 116| 0.373| 0.294| 0.491|
+-------+-----------------+-----------------+------------------+--------------------+--------------------+-------------------+

None
Determing Co-Relation between Input and Output Columns
+------------------+
corr(var_1, label)|
+------------------+
0.9187399607627283|
+------------------+

Correlation between var_1 and label : None
+-------------------+
 corr(var_2, label)|
+-------------------+
0.43652698913681093|
+-------------------+

Correlation between var_2 and label : None
+------------------+
corr(var_3, label)|
+------------------+
0.4014958408311139|
+------------------+

Correlation between var_3 and label : None
+------------------+
corr(var_4, label)|
+------------------+
0.7909100204842113|
+------------------+

Correlation between var_4 and label : None
+------------------+
corr(var_5, label)|
+------------------+
0.7904806260381185|
+------------------+

Correlation between var_5 and label : None

In [3]:
# Feature Engineering
# This is the part where we combine all our input features into a Single Vector using Spark's VectorAssembler
# It creates only a single feature that captures the input values for that Row
# Hence instead of 5 Input Columns it essentially merges all input columns into a Single Feature Vector Column

from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import *
from pyspark.sql.types import *
# We can select the list of the Columns that are to be assembled within the Vector and can pass only those columns to VectorAssembler
print(df.columns)

# Creating a VectorAssembler object by passing the required Input Features
vec_assembler=VectorAssembler(inputCols=['var_1','var_2','var_3','var_4','var_5'],outputCol='features')
# Applying the created vec_assembler over our Input DataFrame by calling transform function
features_df=vec_assembler.transform(df)
print(features_df.printSchema())
print(features_df.show(truncate=False))
# One Additional column named "features" that contains single Dense Vector for all of the inputs
print(features_df.select("features").show(truncate=False))




      

['var_1', 'var_2', 'var_3', 'var_4', 'var_5', 'label']
root
-- var_1: integer (nullable = true)
-- var_2: integer (nullable = true)
-- var_3: integer (nullable = true)
-- var_4: double (nullable = true)
-- var_5: double (nullable = true)
-- label: double (nullable = true)
-- features: vector (nullable = true)

None
+-----+-----+-----+-----+-----+-----+------------------------------+
var_1|var_2|var_3|var_4|var_5|label|features |
+-----+-----+-----+-----+-----+-----+------------------------------+
734 |688 |81 |0.328|0.259|0.418|[734.0,688.0,81.0,0.328,0.259]|
700 |600 |94 |0.32 |0.247|0.389|[700.0,600.0,94.0,0.32,0.247] |
712 |705 |93 |0.311|0.247|0.417|[712.0,705.0,93.0,0.311,0.247]|
734 |806 |69 |0.315|0.26 |0.415|[734.0,806.0,69.0,0.315,0.26] |
613 |759 |61 |0.302|0.24 |0.378|[613.0,759.0,61.0,0.302,0.24] |
748 |676 |85 |0.318|0.255|0.422|[748.0,676.0,85.0,0.318,0.255]|
669 |588 |97 |0.315|0.251|0.411|[669.0,588.0,97.0,0.315,0.251]|
667 |845 |68 |0.324|0.251|0.381|[667.0,845.0,68.0,0.324,0.251]|
758 |890 |64 |0.33 |0.274|0.436|[758.0,890.0,64.0,0.33,0.274] |
726 |670 |88 |0.335|0.268|0.422|[726.0,670.0,88.0,0.335,0.268]|
583 |794 |55 |0.302|0.236|0.371|[583.0,794.0,55.0,0.302,0.236]|
676 |746 |72 |0.317|0.265|0.4 |[676.0,746.0,72.0,0.317,0.265]|
767 |699 |89 |0.332|0.274|0.433|[767.0,699.0,89.0,0.332,0.274]|
637 |597 |86 |0.317|0.252|0.374|[637.0,597.0,86.0,0.317,0.252]|
609 |724 |69 |0.308|0.244|0.382|[609.0,724.0,69.0,0.308,0.244]|
776 |733 |83 |0.325|0.259|0.437|[776.0,733.0,83.0,0.325,0.259]|
701 |832 |66 |0.325|0.26 |0.39 |[701.0,832.0,66.0,0.325,0.26] |
650 |709 |74 |0.316|0.249|0.386|[650.0,709.0,74.0,0.316,0.249]|
804 |668 |95 |0.337|0.265|0.453|[804.0,668.0,95.0,0.337,0.265]|
713 |614 |94 |0.31 |0.238|0.404|[713.0,614.0,94.0,0.31,0.238] |
+-----+-----+-----+-----+-----+-----+------------------------------+
only showing top 20 rows

None
+------------------------------+
features |
+------------------------------+
[734.0,688.0,81.0,0.328,0.259]|
[700.0,600.0,94.0,0.32,0.247] |
[712.0,705.0,93.0,0.311,0.247]|
[734.0,806.0,69.0,0.315,0.26] |
[613.0,759.0,61.0,0.302,0.24] |
[748.0,676.0,85.0,0.318,0.255]|
[669.0,588.0,97.0,0.315,0.251]|
[667.0,845.0,68.0,0.324,0.251]|
[758.0,890.0,64.0,0.33,0.274] |
[726.0,670.0,88.0,0.335,0.268]|
[583.0,794.0,55.0,0.302,0.236]|
[676.0,746.0,72.0,0.317,0.265]|
[767.0,699.0,89.0,0.332,0.274]|
[637.0,597.0,86.0,0.317,0.252]|
[609.0,724.0,69.0,0.308,0.244]|
[776.0,733.0,83.0,0.325,0.259]|
[701.0,832.0,66.0,0.325,0.26] |
[650.0,709.0,74.0,0.316,0.249]|
[804.0,668.0,95.0,0.337,0.265]|
[713.0,614.0,94.0,0.31,0.238] |
+------------------------------+
only showing top 20 rows

None

In [4]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler

# Now we can take only the features column (DenseVector containing our Input Features) and ouput Column and create a Separate DataFrame
model_df=features_df.select("features","label")
print(model_df.show(truncate=False))
print("Checking the Shape of the Model DataSet")
print("Num. of Rows: {0} Num. of Columns: {1}".format(model_df.count(), len(model_df.columns)))
# STEP_5 Splitting the DataSet (Apply randomSplit() function)
"""We have to split out DataSet into training and test dataset in order to train and evaluate the Performance of Linear Regression Model 
We Split it into 70%, 30% ratio to train our Model on 70% of the DataSet. """
train_df,test_df=model_df.randomSplit([0.7,0.3])      

print("Printing Shape of Training and test DataSets")
print("Training DataSet")
print("Num of Rows: {0} Num. of Columns: {1}".format(train_df.count(),len(train_df.columns)))
print("Test DataSet")
print("Num of Rows: {0} Num. of Columns: {1}".format(test_df.count(),len(test_df.columns)))

+------------------------------+-----+
features |label|
+------------------------------+-----+
[734.0,688.0,81.0,0.328,0.259]|0.418|
[700.0,600.0,94.0,0.32,0.247] |0.389|
[712.0,705.0,93.0,0.311,0.247]|0.417|
[734.0,806.0,69.0,0.315,0.26] |0.415|
[613.0,759.0,61.0,0.302,0.24] |0.378|
[748.0,676.0,85.0,0.318,0.255]|0.422|
[669.0,588.0,97.0,0.315,0.251]|0.411|
[667.0,845.0,68.0,0.324,0.251]|0.381|
[758.0,890.0,64.0,0.33,0.274] |0.436|
[726.0,670.0,88.0,0.335,0.268]|0.422|
[583.0,794.0,55.0,0.302,0.236]|0.371|
[676.0,746.0,72.0,0.317,0.265]|0.4 |
[767.0,699.0,89.0,0.332,0.274]|0.433|
[637.0,597.0,86.0,0.317,0.252]|0.374|
[609.0,724.0,69.0,0.308,0.244]|0.382|
[776.0,733.0,83.0,0.325,0.259]|0.437|
[701.0,832.0,66.0,0.325,0.26] |0.39 |
[650.0,709.0,74.0,0.316,0.249]|0.386|
[804.0,668.0,95.0,0.337,0.265]|0.453|
[713.0,614.0,94.0,0.31,0.238] |0.404|
+------------------------------+-----+
only showing top 20 rows

None
Checking the Shape of the Model DataSet
Num. of Rows: 1232 Num. of Columns: 2
Printing Shape of Training and test DataSets
Training DataSet
Num of Rows: 871 Num. of Columns: 2
Test DataSet
Num of Rows: 361 Num. of Columns: 2

In [5]:
# STEP_6: Build and Train Linear Regression Model  (fit(train_df) method of pyspark.ml.regression.LinearRegression)
                                                    model.evaluate(train_df) -> Apply the developed model over the train DataFrame
                                                    modelPrediction1.r2 ->  Retrieve the accuracy % of the model based of training Data
                                                    model.evaluate(test_df) -> Apply the developed  model over the test DataFrame
                                                    modelPrediction2.r2 -> Retrieve the Accuracy % of model based of test Data
                                                    modelPrediction2.meanSquaredError -> Compute the accuracy of the Model 
"""
In this step we build and train the Linear Regression Model using Features of the Input and Output Columns
We can fetch the Co-efficients (B1, B2, B3, B4, B5) for each of the Features and Intercept (B0) values of the model as well
We can also evaluate the Performance of the Model on Training Data as well using r2. 
"""

from pyspark.ml.regression import LinearRegression
lin_Reg=LinearRegression(labelCol="label")

lr_model=lin_Reg.fit(train_df)
print(lr_model.coefficients)
print(lr_model.intercept)
# Evaluating the Model on the Training DataSet
training_predictions=lr_model.evaluate(train_df)
# Fetching the accuracy of the Model on Training Data
print(training_predictions.r2)

# STEP_7: Evaluate Linear Regression Model on Test Data ()
# Evaluating the Model on Test DataSet
test_predictions=lr_model.evaluate(test_df)
# Checking the accuracy of the Model in Test Dataset
print(test_predictions.r2)
# Compute the mean Squared Error 
print("Mean Squared Error: {0}".format(test_predictions.meanSquaredError))

[0.0003494946241817967,4.056115804179784e-05,7.21809797297416e-05,-0.6577943619985317,0.553093216121237]
0.18406129130733442
0.8757714672991658
0.8507603050705955
0.00016073444609295714